#### This code changes the value labels and saves the updated dataframe to a new spsss .sav file

In [12]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    print("Existing value labels for variable:", var_name)
    print(var_metadata)

    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    print("\nUpdated value labels for variable:", var_name)
    print(meta.variable_value_labels[var_name])

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    # pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
    pyreadstat.write_sav(df, new_meta_path)
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

df2 = df[df["Q3_1"] == 1]

# Save the updated metadata to a new file
new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
# pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
pyreadstat.write_sav(df2, new_meta_path)

filtered_df, meta2 = pyreadstat.read_sav(new_meta_path)
meta2_dict = dict(zip(meta2.column_names, meta2.column_labels))


crosstab_result = pd.crosstab(filtered_df['Q3_1']. \
            map(meta2.variable_value_labels['Q3_1']), \
            filtered_df['GENDER'].map(meta2.variable_value_labels['GENDER']), \
            dropna=True, normalize='columns'). \
            loc[meta2.variable_value_labels['Q3_1'].values()]. \
            loc[:,meta2.variable_value_labels['GENDER'].values()]*100


# filter the dataframe to return records that contain 'checked' for Q3_1
# filtered_df = df[df['Q3_1'] == 1]
# print(filtered_df.shape)

# crosstab_result = pd.crosstab(filtered_df['Q3_1'], filtered_df['GENDER'], normalize='columns')
print(crosstab_result)


Existing value labels for variable: GENDER
{1.0: 'Male', 2.0: 'Female', 3.0: 'Other/Non-binary'}

Updated value labels for variable: GENDER
{1.0: 'Male', 2.0: 'Female', 3.0: 'Other_non_binary'}


KeyError: 'Q3_1'

In [ ]:
# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))


# Filter the dataframe to only return Q3_1 == 1
#filtered_df = df[df['Q3_1'] == 1]

pd.crosstab(df['Q3_1'].\
            map(meta.variable_value_labels['Q3_1']), \
            df['GENDER'].map(meta.variable_value_labels['GENDER']), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels['Q3_1'].values()]. \
            loc[:,meta.variable_value_labels['GENDER'].values()]*100

In [2]:
df.shape

(163, 209)

#### This code does the following:
- Reads in an SPSS dataset and metadata
- Modifies variable value labels to get rid of a "/" in the other/binary label
- Creates a dictionary to associate the dataset with the metadata
- Crosstabs a single row variable with a list of column variables
- Concats each crosstab output into a single output dataframe
- Prints the final output dataframe

In [49]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
    # pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
    pyreadstat.write_sav(df, new_meta_path)
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

# list of variables to use as columns and rows for the crosstabs
row_variable = ['Q3_1']
column_variable = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
# variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']

crosstab_result_concat = pd.DataFrame()
# loop through the column variable list and crosstab each one
for col in column_variable:
    crosstab_result = pd.crosstab(df['Q3_1'].\
            map(meta.variable_value_labels['Q3_1']), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels['Q3_1'].values()]. \
            loc[:,meta.variable_value_labels[col].values()]*100
            
    # frames = [crosstab_result_concat, crosstab_result]

    crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    
    print(crosstab_result_concat)


GENDER          Male     Female  Other_non_binary
Q3_1                                             
Unchecked  40.540541  48.863636               0.0
Checked    59.459459  51.136364             100.0
                Male     Female  Other_non_binary      White  Non-White
Q3_1                                                                   
Unchecked  40.540541  48.863636               0.0  47.887324  42.391304
Checked    59.459459  51.136364             100.0  52.112676  57.608696
                Male     Female  Other_non_binary      White  Non-White  \
Q3_1                                                                      
Unchecked  40.540541  48.863636               0.0  47.887324  42.391304   
Checked    59.459459  51.136364             100.0  52.112676  57.608696   

             18 - 34   35 - 54        55+  
Q3_1                                       
Unchecked  53.030303  40.57971  35.714286  
Checked    46.969697  59.42029  64.285714  
                Male     Female  Ot

#### Loop through rows and columns and generating a crosstab for each combination is working

In [67]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
    # pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
    pyreadstat.write_sav(df, new_meta_path)
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

# list of variables to use as columns and rows for the crosstabs
row_variable = ['Q3_1', 'Q3_2']
column_variable = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
# variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']

crosstab_result_concat = pd.DataFrame()
# loop through the column variable list and crosstab each one

# generate crosstabs for each combination of rows and columns
for row in row_variable:

    for col in column_variable:
        crosstab_result = pd.crosstab(df[row].\
            #map(meta.variable_value_labels['Q3_1']), \
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]*100
            
    # frames = [crosstab_result_concat, crosstab_result]

        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    
    print(crosstab_result_concat)


                Male     Female  Other_non_binary      White  Non-White  \
Q3_1                                                                      
Unchecked  40.540541  48.863636               0.0  47.887324  42.391304   
Checked    59.459459  51.136364             100.0  52.112676  57.608696   

             18 - 34   35 - 54        55+  Less thank $40K  $40K - $69.9K  \
Q3_1                                                                        
Unchecked  53.030303  40.57971  35.714286        36.206897      51.219512   
Checked    46.969697  59.42029  64.285714        63.793103      48.780488   

           $70K - $99.9K  $100K+  
Q3_1                              
Unchecked           28.0    60.0  
Checked             72.0    40.0  
                Male     Female  Other_non_binary      White  Non-White  \
Unchecked  40.540541  48.863636               0.0  47.887324  42.391304   
Checked    59.459459  51.136364             100.0  52.112676  57.608696   

             18 - 34   3

In [60]:
import pandas as pd

# Define the dataset
data = {
    'Q3_1': [1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
    'GENDER': [1, 2, 1, 1, 1, 2, 2, 1, 1, 1],
    'AGEGROUP': ['18 - 34', '35 - 54', '18 - 34', '18 - 34', '35 - 54', '55+', '18 - 34', '18 - 34', '35 - 54', '35 - 54']
}

# Create a DataFrame from the dataset
df = pd.DataFrame(data)

# Define the list of rows and columns
row_list = ['Q3_1', 'GENDER']
column_list = ['AGEGROUP']

# Generate crosstabs for each combination of rows and columns
for row in row_list:
    for column in column_list:
        crosstab = pd.crosstab(df[row], df[column])
        print(f'Crosstab for {row} and {column}:\n')
        print(crosstab)
        print('\n')


Crosstab for Q3_1 and AGEGROUP:

AGEGROUP  18 - 34  35 - 54  55+
Q3_1                           
0               2        2    1
1               3        2    0


Crosstab for GENDER and AGEGROUP:

AGEGROUP  18 - 34  35 - 54  55+
GENDER                         
1               4        3    0
2               1        1    1


